In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import h5py

import shutil
from keras.models import load_model
from keras import layers
from keras import models
from keras import optimizers

In [2]:
test_ds = tf.data.Dataset.load('turkey/test')

In [5]:
model1 = load_model('cc_model_with_dropout_2.h5')
model1.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 sequential_8 (Sequential)   (None, 100, 100, 3)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                 

In [13]:
test_predictions = model1.predict(test_ds, batch_size = 32)

326/326 [==============================] - 2s 6ms/step


In [33]:
counter = 0
for element in test_ds:
    preds = model1.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'turkey/test_predictions/undamaged/'
        else:
            dstatus = 'turkey/test_predictions/damaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


In [30]:
counter

10404